In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

def page_text(url):
    site = urlopen("https://www.skku.edu/skku/campus/skk_comm/notice01.do" + url)
    soup = BeautifulSoup(site.read(), "html.parser",from_encoding="utf-8")
    if (soup.find('pre', class_ = "pre") != None):
        page_text = soup.find('pre', class_ = "pre").text
    elif (soup.find('div', class_ = "fr-view") != None):
        page_text = soup.find('div', class_ = "fr-view").text
    else:
        page_text = 'None'
    return page_text


text = [[] for i in range(13260)]
i = 0
while (i <= 13250):
    if (i % 10 == 0):
        site = urlopen("https://www.skku.edu/skku/campus/skk_comm/notice01.do?mode=list&&articleLimit=10&article.offset=" + str(i))
        soup = BeautifulSoup(site.read(), "html.parser",from_encoding="utf-8")
        table = soup.find_all('tr')

    for k in range(1, len(table)):
        sub_table = table[k].find_all('td')
        for j in range(0, len(sub_table) - 1):
            if j != 1:
                text[i].append(sub_table[j].get_text().strip())
            if j == 1:
                if (sub_table[j].find('span') == None):
                    text[i].append('None')
                else:
                    text[i].append(sub_table[j].find('span').string)
                text[i].append(sub_table[j].find('a').string.strip())
                text[i].append(sub_table[j].find('a')["href"])
                text[i].append(page_text(text[i][-1]))
        i += 1

df = pd.DataFrame(text, columns = ['no', 'new', 'title', 'link', 'text', 'date', 'visited'])
df.to_csv('sample3.csv', encoding='utf-8-sig')